In [1]:
import pandas as pd
import docx
import redis
import pyarrow as pa

In [2]:
def parse_document(doc_path, skill_path, tier_start=-1):
    doc = docx.Document(doc_path)
    paras = [x.text for x in doc.paragraphs]
    for i in range(len(paras)):
        if paras[i] == ' ':
            paras[i] = ''
    particular_value = ''
    result = []
    temp_list = []
    for i in paras:
        if i == particular_value:
            temp_list.append(i)
            result.append(temp_list)
            temp_list = []
        else:
            temp_list.append(i)
    result.append(temp_list)
    df = pd.DataFrame({'text':result})
    tier_list = []
    ability_list = []
    tier = tier_start
    for _, row in df.iterrows():
        data = row['text']
        if len(data) > 1:
            if data[0].split(' ')[0] == 'Tier':
                tier += 1
                data.pop(0)
            tier_list.append(tier)
            ability_list.append(data)
    ability_df = pd.DataFrame({'Ability':ability_list, 'Tier':tier_list})
    ability_df['Path'] = skill_path
    ability_df['Skill Name'] = ability_df.Ability.apply(lambda x:x[0].split(':')[0])
    ability_df['Description'] = ability_df.Ability.apply(lambda x:x[0].split(':')[1])
    pr = []
    lim = []
    preq = []
    for _ , row in ability_df.iterrows():
        for sublist in row['Ability']:
            if 'Phys Rep' in sublist:
                pr.append(sublist.split(':')[1])
            if 'Limitations' in sublist:
                lim.append(sublist.split(':')[1])
            if 'Prerequisite' in sublist:
                preq.append(sublist.split(':')[1])
        if 'Phys Rep' not in str(row.Ability):
            pr.append(None)
        if 'Limitations' not in str(row.Ability):
            lim.append(None)
        if 'Prerequisite' not in str(row.Ability):
            preq.append(None)
    ability_df['Phys Rep'] = pr
    ability_df['Limitations'] = lim
    ability_df['Prerequisite'] = preq
    ability_df = ability_df.map(lambda x: x.strip() if isinstance(x, str) else x)
    return ability_df[['Skill Name', 'Description', 'Path', 'Tier', 'Limitations', 'Phys Rep', 'Prerequisite']]

In [3]:
warrior_df = parse_document("The Warrior's Path.docx", 'Warrior')
rogue_df = parse_document("The Rogue's Path.docx", 'Rogue')
healer_df = parse_document("The Healer's Path.docx", 'Healer')
mage_df = parse_document("The Mage's Path.docx", 'Mage')
bard_df = parse_document("The Bard's Path.docx", 'Bard', tier_start=0)

In [4]:
skills = pd.concat([warrior_df,rogue_df,healer_df,mage_df,bard_df])

In [5]:
skills[skills.Path == 'Bard']

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite
0,Field Instrument,The Bard is able to carry an instrument of the...,Bard,1,None,None,None
1,Song of Surrender,The Bard has the ability to placate any foe te...,Bard,1,Can only be used twice per day with an additio...,The bard must play a song for a specific oppon...,None
2,Surf,The Bard plays a tune that resonates with the ...,Bard,1,Can only be used once per day with an addition...,"A song, lasting while surfing, played for the ...",None
3,Bardic Inspiration,The Bard plays an inspirational tune that give...,Bard,2,The additional armor point only lasts only as ...,A 15 second song intro is played to a specific...,None
4,Healing Song,The Bard is able to channel healing magic thro...,Bard,2,Can only be used twice per day with an additio...,"A song, lasting 30 seconds, played for a speci...",None
5,The Right Key,The Bard is able to play a tune followed by a...,Bard,2,Can only be used once a day plus plus once per...,A series of notes played on their instrument e...,None
6,Party Inspiration,The Bard plays an inspirational tune that give...,Bard,3,The additional armor point only lasts only as ...,A 30 second song intro is played to a group of...,Bardic Inspiration
7,Surf Party,The Bard plays a tune that resonates with the ...,Bard,3,Can only be used once per day with an addition...,"A song, lasting while surfing, played for the ...",Surf
8,Curative Works,The Bard is able to channel more powerful heal...,Bard,4,Can only be used once per day with an addition...,"A song, lasting 30 seconds, played for specifi...",Healing Song
9,Song of Sight,The Bard is able to play a melody that resonat...,Bard,4,Can only be used once per day with an addition...,"A song, lasting 30 seconds, played towards the...",None


In [6]:
skills.to_excel('Skills_Table.xlsx', index=False)

In [7]:
skills.to_csv('Skills_Table.csv', index=False)

In [8]:
from math import floor, sqrt
def tier(events):
    return floor((sqrt(8*events)-1)/2)